In [18]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry

from api_key import APIKEY

API_KEY = APIKEY

In [50]:
import pandas as pd

df = pd.read_csv('/Users/moinkhwaja/Documents/GitHub/Heterojunction-Metal-Organic-Framework-Photocataysts/Metal_Oxide_Method/HSE_database_method/HCOOH_pass.csv')
mpid = df['MPID'].tolist()
print(len(mpid))


13


In [51]:
CO = ['mp-3414', 'mp-542001', 'mp-28845', 'mp-561423', 'mp-14276', 'mp-29590', 'mp-556442', 'mp-27980']
CH4 = ['mp-680722', 'mp-31107', 'mp-1095378', 'mp-23569', 'mp-30003', 'mp-1190864', 'mp-23084', 'mp-15511', 'mp-7128', 'mp-28855', 'mp-561561', 'mp-1079364', 'mp-542115', 'mp-31232', 'mp-755116', 'mp-1091363', 'mp-558910', 'mp-14037', 'mp-29698', 'mp-1095283', 'mp-20805', 'mp-1113861', 'mp-16834', 'mp-778924', 'mp-556120', 'mp-20727', 'mp-14368', 'mp-1819', 'mp-22467', 'mp-753747', 'mp-15750', 'mp-554435', 'mp-1105700', 'mp-1217378', 'mp-680219', 'mp-17637', 'mp-759342', 'mp-23446', 'mp-779156', 'mp-17677', 'mp-685353', 'mp-230', 'mp-31069', 'mp-1111368', 'mp-759307', 'mp-624234', 'mp-10798', 'mp-760188', 'mp-7094']
CHOOH = ['mp-20015', 'mp-14367', 'mp-541114', 'mp-28299', 'mp-562338', 'mp-554944', 'mp-768936', 'mp-29193']


In [52]:
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in mpid:
        material_entries = mpr.materials.search(material_ids=[mp_id])

        if material_entries:
            mp_entries.append(material_entries[0])

chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')]
materials_gga = [material + "-GGA" for material in mpid]
materials_id = mpid


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
from pymatgen.core.composition import Composition

def remove_o_and_h(composition):
    new_composition_elements = []
    for element, count in composition.get_el_amt_dict().items():
        if element not in ['O', 'H']:
            new_composition_elements.append(f"{element}{int(count)}")
    new_composition_str = ' '.join(new_composition_elements)
    return Composition(new_composition_str)

filtered_molecules = [remove_o_and_h(molecule) for molecule in composition_list]

In [54]:
from pymatgen.core.composition import Composition

def calculate_percent_composition(molecule):
    total_atoms = sum(molecule.values())
    return {element.symbol: count / total_atoms for element, count in molecule.items()}

percent_compositions = [calculate_percent_composition(molecule) for molecule in filtered_molecules]

In [55]:
pbx_data = []
pbx_entries = []

total_iterations = len(chemsys_list)  

with MPRester(API_KEY) as mpr:
    for index, (i, x) in enumerate(zip(chemsys_list, percent_compositions)):
        try:
            pourbaix_entries = mpr.get_pourbaix_entries(i)
            pbx_entries.append(pourbaix_entries)
            pbx_diagram = PourbaixDiagram(entries=pourbaix_entries, comp_dict=x)
            pbx_data.append(pbx_diagram)
        except Exception as e:
            pbx_data.append('error')
        
        percent_complete = ((index + 1) / total_iterations) * 100
        print(f'Percent complete: {percent_complete:.2f}%')

Retrieving ThermoDoc documents:   0%|          | 0/243 [00:00<?, ?it/s]

Percent complete: 7.69%


Retrieving ThermoDoc documents:   0%|          | 0/260 [00:00<?, ?it/s]

Percent complete: 15.38%


Retrieving ThermoDoc documents:   0%|          | 0/270 [00:00<?, ?it/s]

...

Percent complete: 84.62%


Retrieving ThermoDoc documents:   0%|          | 0/523 [00:00<?, ?it/s]

Percent complete: 92.31%


Retrieving ThermoDoc documents:   0%|          | 0/259 [00:00<?, ?it/s]

Percent complete: 100.00%


In [56]:
matched_entries = []

for mat_id, entries in zip(materials_gga, pbx_entries):
    matched_entry = next((entry for entry in entries if entry.entry_id == mat_id), None)
    
    if matched_entry:
        matched_entries.append(matched_entry)
    else:
        matched_entries.append(None)

In [57]:
material_stabilities = []

for i, x in zip(pbx_data, matched_entries):
    stable_count = 0
    total_pH_count = 0

    for pH in range(3, 13):
        try:
            stability = i.get_decomposition_energy(x, pH, 0)
            total_pH_count += 1
            if stability < 0.5:
                stable_count += 1
        except Exception as e:
            continue
    
    if total_pH_count > 0:
        stability_percentage = (stable_count / total_pH_count) * 100
    else:
        stability_percentage = 0

    material_stabilities.append({'entry': x, 'stability_percentage': stability_percentage})

for material in material_stabilities:
    print(f"Material: {material['entry']}, Stability Percentage: {material['stability_percentage']:.2f}%")


Material: PourbaixEntry(Cd4 Ge4 Bi8 O24 with energy = -4.7746, npH = -48.0, nPhi = -48.0, nH2O = 24.0, entry_id = 'mp-555027-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(Na1 U1 F6 with energy = -28.2592, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-1205997-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(Zn4 Hg4 O2 F12 with energy = -34.6462, npH = -4.0, nPhi = -4.0, nH2O = 2.0, entry_id = 'mp-17972-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(Te4 Mo6 Se14 Br20 with energy = -34.1764, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-645663-GGA'), Stability Percentage: 70.00%
Material: PourbaixEntry(Cs2 Ag1 Sb1 Cl6 with energy = -14.4806, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-1112940-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(Rb4 U2 F12 with energy = -60.9994, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-13595-GGA'), Stability Percentage: 80.00%
Material: PourbaixEntry(Ti4 Tl16 Se16 with energy = -31.8709, npH = 0, nPhi = 0, n

In [58]:
ph7 = []

for i, x in zip(pbx_data, matched_entries):
    stable_count = 0
    total_pH_count = 0

    
    stability = i.get_decomposition_energy(x, 7, 0)
    ph7.append(stability)

ph7_str = ', '.join(str(value) for value in ph7)
print(ph7_str)

        
    

0.036377836553505995, 0.26727524608542774, 0.39107713929266735, 0.23358697356677638, 0.3075803642718309, 0.36691550255699923, 0.3674515199880303, 0.27423687888423987, 0.49873885798453, 0.2967020702614761, 0.22261250432878318, 0.12334245690157175, 0.13739272604087382
